In [ ]:
import ccxt
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import asyncio
import websockets
import json
import config
from phemex import *

In [ ]:
# Set up CCXT Phemex API
exchange = initExchange()
# exchange = ccxt.phemex()

# Define parameters for diamond pattern
n_highs = 4  # number of highs
n_lows = 4  # number of lows
tolerance = 0.05  # percentage tolerance for trendline convergence
min_height = 0.05  # minimum height of diamond pattern
max_height = 0.5  # maximum height of diamond pattern

# Get OHLCV data for a specific trading pair
symbol = 'BTC/USD'
timeframe = '1d'
limit = 1000
ohlcv_data = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=limit)
df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Define function to detect diamond pattern
def detect_diamond(df, n_highs, n_lows, tolerance, min_height, max_height):
    highs = df['high'].rolling(n_highs).max().dropna()
    lows = df['low'].rolling(n_lows).min().dropna()

    if len(highs) == 0 or len(lows) == 0:
        return False

    # Check if there are enough highs and lows for a diamond pattern
    if len(highs) < n_highs or len(lows) < n_lows:
        return False

    # Find points of convergence for the trendlines
    convergence_points = []
    for i in range(n_highs - 1, len(highs)):
        for j in range(n_lows - 1, len(lows)):
            if abs((highs.iloc[i] - lows.iloc[j]) / highs.iloc[i]) <= tolerance:
                convergence_points.append((highs.index[i], lows.index[j]))

    # Check if there are any convergence points
    if len(convergence_points) == 0:
        return False

    # Calculate the height of the diamond pattern
    heights = []
    # display(df)
    # print(convergence_points)
    for point in convergence_points:
        print('point 1 and 2',point[1],point[0])
        print('max',df.loc[point[1]:point[0], 'high'].max())
        print('min',df.loc[point[1]:point[0], 'low'].min())
        height = df.loc[point[1]:point[0], 'high'].max() - df.loc[point[1]:point[0], 'low'].min()
        height_pct = height / df.loc[point[1]:point[0], 'close'].iloc[-1]
        heights.append(height_pct)

    if len(heights) == 0:
        return False

    max_height_idx = np.argmax(heights)
    max_height_val = heights[max_height_idx]
    max_height_point = convergence_points[max_height_idx]

    if max_height_val < min_height or max_height_val > max_height:
        return False

    # Check if diamond pattern is bullish or bearish
    if df.loc[max_height_point[1], 'close'] < df.loc[max_height_point[0], 'close']:
        return 'bearish'
    else:
        return 'bullish'

# Detect diamond pattern in OHLCV data
diamond_pattern = detect_diamond(df, n_highs, n_lows, tolerance, min_height, max_height)

if diamond_pattern:
    print(f'Diamond pattern detected: {diamond_pattern}')
else:
    print('No diamond pattern detected.')


In [ ]:
def getTimeformat(interval):
    if 'S' in interval:
        timeformat = "%Y-%m-%d, %H:%M:%S"
    elif 'min' in interval:
        timeformat = "%Y-%m-%d, %H:%M"
    elif 'H' in interval:
        timeformat = "%Y-%m-%d, %H"    
    elif 'D' in interval:
        timeformat = "%Y-%m-%d"  
    elif 'M' in interval:
        timeformat = "%Y-%m"     
    elif 'Y' in interval:
        timeformat = "%Y"  
    else:
        timeformat = "%Y-%m-%d, %H:%M:%S"
    return timeformat

In [ ]:
async def getOhlc(trades,interval='1min'):
    global ohlcvData
    try:   
        # print('-- Function getOhlc --')
        ohlc = pd.DataFrame(trades,columns=['open','high','low','close','volume','epoch'])
        ohlc = trades['priceEp'].resample(interval).ohlc()
        tick_datetime_object = pd.to_datetime(ohlc.index, unit='ns',utc=True)
        timeformat = getTimeformat(interval)
        timenow = pd.to_datetime(pd.to_datetime(tick_datetime_object).strftime(timeformat))
        ohlc['epoch'] = timenow.astype(np.int64) / 10**9    
        ohlc['volume'] = trades['volume'].resample(interval).mean() 
        # ohlc = await getVolumeColorDf(ohlc)
        ohlc = ohlc[ohlc['open'] > 0]
        ohlc.dropna()
        ohlcvData = pd.concat([ohlcvData,ohlc])
        return ohlcvData
    except Exception as e:
            print('Foutmelding in functie getOhlc: {}'.format(e))  

In [ ]:
async def getTradePrices(message,interval):
    # print('-- Function getTradePrices --')
    global trades
    try:        
        if len(trades) > 0:
            trades.append(json.loads(message)['trades'][0])
        else:
            trades = list(json.loads(message)['trades'])
        df = pd.DataFrame(trades,columns=['timestamp','side','priceEp','volume'])
        tick_datetime_object = pd.to_datetime(df['timestamp'], unit='ns',utc=True)
        timenow = pd.to_datetime(tick_datetime_object + timedelta(hours=1))
        df['timestamp'] = timenow
        df.set_index('timestamp',inplace=True)
        df.reset_index()
        tradesAgg = df.groupby([pd.Grouper(level='timestamp', freq=interval)]).agg({'side':lambda x: list(x),'priceEp':'max','volume':'sum'})
        
        return tradesAgg.dropna()
    except Exception as e:
            print('Foutmelding in functie getTradePrices: {}'.format(e))

In [ ]:
async def send_message(message):
    try:
        global ohlcvData, tradesPrices,mbtcTickerData,clients,sendMessage,orderbookData,candlestickData
        if message != None:
            if 'trades' in message:
                tradesPrices= await getTradePrices(message,'1S')
            # print('len(tradesPrices)',len(tradesPrices))
            if len(tradesPrices) > 300:
                clientsCopy = clients.copy()
                for cid in clientsCopy.keys():
                    try:
                        # print('Client Interval',clientsCopy[cid]['interval'])
                        ohlcvData = await getOhlc(tradesPrices,clientsCopy[cid]['interval'])
                        
                    except websockets.ConnectionClosed:
                        continue
    except Exception as e:
        print('Error in send_message: {}'.format(e))

In [ ]:
# connect to Phemes websocket server
async def connectPhemexWS():
    global interval
    async for websocket in websockets.connect('wss://vapi.phemex.com/ws'):
        try:
            print('Phemex Websocket Server is connected!')
            subscribe_ticker_msg = json.dumps({
                "id": 0,
                "method": "tick.subscribe",
                "params": [config.MBTC_SYMBOL]
            })
            await websocket.send(subscribe_ticker_msg)
            
            subscribe_Trade_msg = json.dumps({
                "id": 0,
                "method": "trade.subscribe",
                "params": [config.TRADE_SYMBOL]
            })
            await websocket.send(subscribe_Trade_msg)  

            subscribe_orderbook_msg = json.dumps({
                "id": 0,
                "method": "orderbook.subscribe",
                "params": [config.TRADE_SYMBOL,True]
            })
            await websocket.send(subscribe_orderbook_msg)              
                     
            while True:
                message = await websocket.recv() 
                # if 'book' in message:
                #     print(message)          
                await send_message(message)
        except websockets.ConnectionClosed:
            continue

In [ ]:
# start python localhost websocket server
async def start_server():
    print('Server started')
    try:
        # await websockets.serve(register_new_client,"localhost",3000)
        await connectPhemexWS()
    except Exception as e:
        print('Error {}'.format(e))

In [ ]:
if __name__ == '__main__':
    event_loop = asyncio.get_event_loop()
    event_loop.run_until_complete(start_server())
    event_loop.run_forever()